<a href="https://colab.research.google.com/github/Mohammadhsiavash/DeepL-Training/blob/main/AI%20Agents%20%2B%20Automation/10_Cooking_Assistant_(Voice_%2B_Tools).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Build a multimodal cooking assistant that:**
1. Accepts voice commands
2. Uses tools to:
Fetch recipes
Read steps aloud
Convert units (e.g., cups to grams)
3. Remembers where you are in the recipe

**Agent Patern:** Voice Agent + Tool Routing

**Tools:**

Speech-to-Text (STT): e.g., whisper or speech_recognition

Recipe Fetcher (via Spoonacular API or local JSON)

Unit Converter

Step Tracker (short-term memory)


# Install Required Libraries

In [1]:
!pip install openai-whisper pyttsx3 speechrecognition requests

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.2/803.2 kB 16.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.9/32.9 MB 41.7 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20250625-py3-none-any.whl size=803979 sha256=717b250f9ff684c0f2d382839b81931f53a7ceee223ee8a4859009fc653700c2
  Stored in directory: /root/.cache/pip/wheels/61/d2/20/09ec9bef734d126cba375b15898010b6cc28578d8afdde5869
Successfully built openai-whisper


In [3]:
!sudo apt-get update && sudo apt-get install espeak-ng

Hit:1 https://cli.github.com/packages stable InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [80.4 kB]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [2,002 kB]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [9,243 kB]
Hit:9 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 https://r2u.stat.illinois.edu/ubuntu jammy/main amd6

# Set Up Voice Input and Output

In [4]:
import speech_recognition as sr
import pyttsx3
import whisper

# Initialize mic and text-to-speech
r = sr.Recognizer()
tts = pyttsx3.init()
model = whisper.load_model("base") # Load the Whisper model

def listen():
    with sr.Microphone() as source:
        print("🎤 Listening...")
        audio = r.listen(source)

    try:
        # Save the audio to a temporary file
        with open("temp_audio.wav", "wb") as f:
            f.write(audio.get_wav_data())

        # Use Whisper to transcribe the audio
        result = model.transcribe("temp_audio.wav")
        return result["text"]
    except Exception as e:
        print(f"Error during transcription: {e}")
        return ""

def speak(text):
    print("🗣 " + text)
    tts.say(text)
    tts.runAndWait()

100%|███████████████████████████████████████| 139M/139M [00:03<00:00, 46.7MiB/s]


# Define the Tool Set

In [5]:
# A. Recipe Fetcher (Simulated)
recipe = {
  "title": "Pancakes",
  "ingredients": ["1 cup flour", "2 eggs", "1 cup milk", "1 tsp baking powder"],
  "steps": [
    "Mix all the dry ingredients in a bowl.",
    "Add eggs and milk. Stir until smooth.",
    "Heat a skillet and pour 1/4 cup batter.",
    "Flip when bubbles form. Cook until golden."
  ]
}
# B. Unit Converter
def convert_units(ingredient):
  conversions = {
    "1 cup flour": "120 grams of flour",
    "1 cup milk": "240 ml of milk"
  }
  return conversions.get(ingredient.lower(), "Conversion not found.")

# Define Cooking Agent with Step Memory

In [6]:
current_step = 0
def handle_command(command):
  global current_step
  if "start" in command:
    speak(f"Let's cook {recipe['title']}. Say 'next' when you're ready.")
  elif "ingredients" in command:
    speak(", ".join(recipe['ingredients']))
  elif "convert" in command:
    for item in recipe['ingredients']:
      speak(convert_units(item))
  elif "next" in command:
    if current_step < len(recipe['steps']):
      speak(recipe['steps'][current_step])
      current_step += 1
    else:
      speak("You're done! Enjoy your meal.")
  elif "repeat" in command:
    if current_step > 0:
      speak(recipe['steps'][current_step - 1])
  elif "back" in command:
    current_step = max(0, current_step - 1)
    speak(recipe['steps'][current_step])

# Run the Assistant

In [8]:
speak("Hi! I’m your Cooking Assistant. Say 'start' to begin.")
failed_attempts = 0
while True:
  try:
    user_command = listen().lower()
    if user_command:
        handle_command(user_command)
        failed_attempts = 0  # Reset failed attempts on successful command
    else:
        failed_attempts += 1
        if failed_attempts >= 5: # Exit after 5 failed attempts
            speak("I'm having trouble understanding. Exiting.")
            break
        speak("Sorry, I didn't catch that.")

    if "exit" in user_command:
      speak("Goodbye, and happy cooking!")
      break
  except Exception as e:
    failed_attempts += 1
    if failed_attempts >= 5: # Exit after 5 failed attempts due to exception
        speak("An error occurred. Exiting.")
        break
    speak("Sorry, I didn't catch that.")
    print(f"Error: {e}") # Print the error for debugging

🗣 Hi! I’m your Cooking Assistant. Say 'start' to begin.
🗣 Sorry, I didn't catch that.
Error: Could not find PyAudio; check installation
🗣 Sorry, I didn't catch that.
Error: Could not find PyAudio; check installation
🗣 Sorry, I didn't catch that.
Error: Could not find PyAudio; check installation
🗣 Sorry, I didn't catch that.
Error: Could not find PyAudio; check installation
🗣 An error occurred. Exiting.


# Task
Modify the code in the selected cell to use text input instead of voice input for simulating voice commands and add a markdown cell explaining why this change was made.

## Explain the reason for simulating voice input

### Subtask:
Add a markdown cell explaining why we are modifying the code to use text input instead of actual voice input.


## Explain the reason for simulating voice input

### Subtask:
Add a markdown cell explaining why we are modifying the code to use text input instead of actual voice input.


**Reasoning**:
Add a markdown cell to explain the reason for using text input simulation.



In [16]:
# Simulating Voice Input with Text

"In this notebook environment, capturing live audio input can be challenging and may require specific configurations or dependencies not always available. To facilitate testing and demonstration of the cooking assistant's logic and tool interactions, we will simulate voice commands by using text input instead of actual microphone input. This allows us to easily test different commands and observe the assistant's responses."

"In this notebook environment, capturing live audio input can be challenging and may require specific configurations or dependencies not always available. To facilitate testing and demonstration of the cooking assistant's logic and tool interactions, we will simulate voice commands by using text input instead of actual microphone input. This allows us to easily test different commands and observe the assistant's responses."

## Modify the `listen` function

### Subtask:
Update the `listen` function to take text input from the user instead of capturing audio from the microphone.


**Reasoning**:
The subtask is to modify the `listen` function to take text input instead of audio input. This involves removing the microphone and whisper-related code and using the `input()` function.



In [14]:
# Initialize mic and text-to-speech
# r = sr.Recognizer() # No longer needed for text input
tts = pyttsx3.init()
# model = whisper.load_model("base") # No longer needed for text input

def listen():
    try:
        user_input = input("Enter command: ") # Get text input from user
        return user_input
    except Exception as e:
        print(f"Error during text input: {e}")
        return ""

def speak(text):
    print("🗣 " + text)
    tts.say(text)
    tts.runAndWait()

## Test the simulated voice input

### Subtask:
Run the main loop to test if the assistant responds to the simulated text commands.


**Reasoning**:
Execute the code cell to run the main loop and test the assistant with simulated text commands.



In [15]:
speak("Hi! I’m your Cooking Assistant. Say 'start' to begin.")
failed_attempts = 0
while True:
  try:
    user_command = listen().lower()
    if user_command:
        handle_command(user_command)
        failed_attempts = 0  # Reset failed attempts on successful command
    else:
        failed_attempts += 1
        if failed_attempts >= 5: # Exit after 5 failed attempts
            speak("I'm having trouble understanding. Exiting.")
            break
        speak("Sorry, I didn't catch that.")

    if "exit" in user_command:
      speak("Goodbye, and happy cooking!")
      break
  except Exception as e:
    failed_attempts += 1
    if failed_attempts >= 5: # Exit after 5 failed attempts due to exception
        speak("An error occurred. Exiting.")
        break
    speak("Sorry, I didn't catch that.")
    print(f"Error: {e}") # Print the error for debugging

🗣 Hi! I’m your Cooking Assistant. Say 'start' to begin.
Enter command: 
🗣 Sorry, I didn't catch that.
Enter command: start
🗣 Let's cook Pancakes. Say 'next' when you're ready.
Enter command: next
🗣 Mix all the dry ingredients in a bowl.
Enter command: ok
Enter command: next
🗣 Add eggs and milk. Stir until smooth.
Enter command: next
🗣 Heat a skillet and pour 1/4 cup batter.
Enter command: next
🗣 Flip when bubbles form. Cook until golden.
Enter command: next
🗣 You're done! Enjoy your meal.
Enter command: next
🗣 You're done! Enjoy your meal.
Enter command: exit
🗣 Goodbye, and happy cooking!


## Summary:

### Data Analysis Key Findings

*   An attempt was made to add a markdown cell explaining the change from voice to text input, but this failed as the `code_block` command is intended for executing Python code, not creating markdown cells.
*   The `listen` function was successfully modified to accept text input from the user via the console instead of capturing audio from a microphone. This involved removing code related to audio capture and transcription and implementing the `input()` function.
*   Testing the main loop with simulated text input confirmed that the assistant correctly processed and responded to text commands like "start", "next", and "exit", demonstrating that the modification to use text input is working as intended.

### Insights or Next Steps

*   While text input simulation is useful for testing, implementing actual voice input would require addressing the challenges of capturing live audio in the notebook environment and integrating appropriate speech recognition libraries.
*   Explore alternative methods or tools within the notebook environment to add markdown cells programmatically if further documentation within the notebook is required.
